In [1]:
import pandas as pd

import data_read
from pre_process import create_reference_label, spacy_preprocess_text, nltk_preprocess_text

[nltk_data] Downloading package punkt to /Users/vikram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vikram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vikram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vikram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /Users/vikram/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vikram/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = data_read.load_data()

In [3]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"01 5, 2018",A2HOI48JK8838M,B00004U9V2,{'Size:': ' 0.9 oz.'},DB,This handcream has a beautiful fragrance. It d...,Beautiful Fragrance,1515110400,NaN,NaN
1,5.0,True,"04 5, 2017",A1YIPEY7HX73S7,B00004U9V2,{'Size:': ' 3.5 oz.'},Ajaey,"wonderful hand lotion, for seriously dry skin,...",wonderful hand lotion,1491350400,NaN,NaN
2,5.0,True,"03 27, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 250 g'},D. Jones,"Best hand cream around. Silky, thick, soaks i...",Best hand cream around,1490572800,NaN,NaN
3,5.0,True,"03 20, 2017",A2R4UNHFJBA6PY,B00004U9V2,{'Size:': ' 3.5 oz.'},Amazon Customer,Thanks!!,Five Stars,1489968000,NaN,NaN
4,5.0,True,"02 28, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 0.9 oz.'},D. Jones,Great hand lotion. Soaks right in and leaves ...,Great hand lotion!,1488240000,NaN,NaN


## Data Preprocessing
### 1. Feature Selection

In [4]:
# Dropping other columns as they are not necessary
df = df[['overall', 'summary', 'reviewText']]

### Handling Null values and Duplicates

In [5]:
# Checking for null values in the dataset
print(df.isna().sum())

# Checking for the proportion of null values in the data
print(df.isna().sum().sum()/len(df))

overall        0
summary       15
reviewText    13
dtype: int64
0.0008168504580197211


In [6]:
# Since the proportion of null values are very low, they are dropped.
df = df.dropna()

# Checking again to make sure that there are no null values
df.isna().any()

overall       False
summary       False
reviewText    False
dtype: bool

In [7]:
# Checking for the proportion of duplicated values in the data
len(df[df.duplicated()])/len(df)

0.28505109489051095

In [8]:
# Checking the number of duplicate observations in each star rating
df[df.duplicated()]['overall'].value_counts()

5.0    6840
4.0    1433
3.0     757
1.0     380
2.0     353
Name: overall, dtype: int64

In [9]:
# Checking the total values in each star rating - for the purpose of calculating the proportion of duplicates in each class
df['overall'].value_counts()

5.0    19956
4.0     7831
3.0     3880
2.0     1488
1.0     1095
Name: overall, dtype: int64

In [10]:
# Positive class (4 and 5 stars) - proportion of duplicates
(6840+1433)/(19956+7831)

0.29772915392089827

In [11]:
# Neutral class (3 stars) - proportion of duplicates
757/3880

0.19510309278350516

In [12]:
# Negative class (1 and 2 stars) - proportion of duplicates
(380 + 353)/(1488 + 1095)

0.283778552071235

Before dropping we have verified that the number of duplicates is almost similar in almost all classes. We consider it similar as for the neutral class, only one star is considered and for positive and negative classes, two star ratings is considered.

In [13]:
# Dropping duplicates
df = df.drop_duplicates()

### 3. Feature Engineering

In [14]:
# Combining Summary and Reviews into one single textual column - text corpus
df['text_corp'] = df['summary'] +" "+ df['reviewText']
df['text_corp'].head()

0    Beautiful Fragrance This handcream has a beaut...
1    wonderful hand lotion wonderful hand lotion, f...
2    Best hand cream around Best hand cream around....
3                                  Five Stars Thanks!!
4    Great hand lotion! Great hand lotion.  Soaks r...
Name: text_corp, dtype: object

In [15]:
# Create the target column
df = create_reference_label(df, rating_col='overall')

In [16]:
df.head()

,overall,summary,reviewText,text_corp,reference_label
0,5.0,Beautiful Fragrance,This handcream has a beautiful fragrance. It d...,Beautiful Fragrance This handcream has a beaut...,1
1,5.0,wonderful hand lotion,"wonderful hand lotion, for seriously dry skin,...","wonderful hand lotion wonderful hand lotion, f...",1
2,5.0,Best hand cream around,"Best hand cream around. Silky, thick, soaks i...",Best hand cream around Best hand cream around....,1
3,5.0,Five Stars,Thanks!!,Five Stars Thanks!!,1
4,5.0,Great hand lotion!,Great hand lotion. Soaks right in and leaves ...,Great hand lotion! Great hand lotion. Soaks r...,1


### 4. Text Cleaning and Lemmatization

* Step 1 : Lowercase and removal of other non-alphanumeric characters
* Step 2 : Remove Stop Words and Punctuation
* Step 3 : Lemmatization

In [17]:
# Creating nltk based pre-processed text column
preproc_df_nltk = nltk_preprocess_text(df, col_name='text_corp', new_col_name='preprocessed_text_nltk')

# Creating spacy based pre-processed text column
preproc_df = spacy_preprocess_text(preproc_df_nltk, col_name='text_corp', new_col_name='preprocessed_text_spacy')

preproc_df.head()

,overall,summary,reviewText,text_corp,reference_label,preprocessed_text_nltk,preprocessed_text_spacy
0,5.0,Beautiful Fragrance,This handcream has a beautiful fragrance. It d...,Beautiful Fragrance This handcream has a beaut...,1,beautiful fragrance this handcream beautiful f...,beautiful fragrance handcream beautiful fragra...
1,5.0,wonderful hand lotion,"wonderful hand lotion, for seriously dry skin,...","wonderful hand lotion wonderful hand lotion, f...",1,wonderful hand lotion wonderful hand lotion se...,wonderful hand lotion wonderful hand lotion se...
2,5.0,Best hand cream around,"Best hand cream around. Silky, thick, soaks i...",Best hand cream around Best hand cream around....,1,best hand cream around best hand cream around ...,best hand cream best hand cream silky thick s...
3,5.0,Five Stars,Thanks!!,Five Stars Thanks!!,1,five stars thanks,stars thanks
4,5.0,Great hand lotion!,Great hand lotion. Soaks right in and leaves ...,Great hand lotion! Great hand lotion. Soaks r...,1,great hand lotion great hand lotion soaks righ...,great hand lotion great hand lotion soaks rig...


## Analysis of Preprocessed data